Below, I use data analysis and manipulation tools on data from Bellabeat, a high-tech manufacturer of health products for women. This is my second case study from the Google Data Analytics Professional Certificate.
My intention is to isolate insights on what the data says about the state of the health of the users, and what they can do to improve their health. By showing the users how valuable Bellabeat tools are, I intend to convince them to use these tools even more. These insights can also be used as advertisement highlights to convince new customers to adopt Bellabeat's tools.  

In [ ]:
library("tidyverse")

In [ ]:
#Rename files in Kaggle input directory 
daily_activity <- read.csv("/kaggle/input/bellabeat-datasets/dailyActivity_merged.csv")
sleep_day <- read.csv("/kaggle/input/bellabeat-datasets/sleepDay_merged.csv")

daily_activity table is a dataset containing health metrics obtained from Bellabeat users over the course of each day. It measures actions like distance steps taken, distance moved and calories consumed. 

sleep_day dataset records the users' sleep, noting how many sessions of sleep they have per day, how many minutes asleep and how much time in bed. 

In [ ]:
#Let's take a look at both datasets. 
head(daily_activity)
head(sleep_day)

We can also check the column names, and how many distinct IDs there are in both datasets. 

In [ ]:
#To check column names 
colnames(daily_activity)
colnames(sleep_day)

#To check distinct IDs
n_distinct(daily_activity$Id)
n_distinct(sleep_day$Id)

In both datasets, I'm more interested in particular columns and below I'll take summary statistics of these columns. 

In [ ]:
#summarize selected columns in the daily_activity dataset. 
daily_activity %>%
  select(TotalSteps,
         TotalDistance,
         SedentaryMinutes) %>%
  summary()

#summarize selected columns in the sleep_day dataset
sleep_day %>%
  select(TotalSleepRecords,
         TotalMinutesAsleep,
         TotalTimeInBed) %>%
  summary()

The sleep_day dataset separates the time asleep and time in bed measurements. Naturally, I expect these datasets to be equal or to closely correlate. Let's attempt to check how correlated they are. 

In [ ]:
#To check correlation between TotalMinutesAsleep and TotalTimeInBed
cor(sleep_day$TotalMinutesAsleep, sleep_day$TotalTimeInBed)

A Pearson's correlation figure of 0.93 shows a strong positive relationship between TotalMinutesAsleep and TotalTimeInBed. My postulation was correct. 
We can further draw a graph to show this correlation. 

In [ ]:
#Draw a graph of TotalTimeInBed against TotalMinutesAsleep
ggplot(data=sleep_day, aes(x=TotalMinutesAsleep, y=TotalTimeInBed)) + geom_point() + geom_smooth(method = "lm", se = TRUE, color = "red") + labs(title = "Strong Positive Correlation (r = 0.93)", TotalMinutesAsleep = "X variable", TotalTimeInBed = "Y variable")

In further analyzing both datasets, I intend to aggregate data by dates, and day of the week. First I have to confirm the data type for the dates in both datasets. 

In [ ]:
#confirm the datatype for sleepDay in sleep_day dataset
class(sleep_day$SleepDay)

The first step is creating a dateonly column in the sleep_day dataset by removing the time component from all the SleepDay entries. 

In [ ]:
#first step of extracting the date alone from the dateonly column. Data cleaning. 
sleep_day$dateonly <- sub(" \\d{2}:\\d{2}:\\d{2} (AM|PM)", "", sleep_day$SleepDay)

#take a new look at the sleep_day dataset
head(sleep_day)

The new column dateonly has to be formated to date datatype so it can be used in further analysis. This is achieved by introducing a new column called sleepdate

In [ ]:
#create sleepdate as the date component of dateonly
sleep_day$sleepdate<-as.Date(sleep_day$dateonly, format = "%m/%d/%Y")

#take a new view of sleep_day
head(sleep_day)

To generate more insights, I'll identify the day of the week for each day in sleepday by introducing a new column called day_of_week. 

In [ ]:
#identify the day of the week for each date in sleep day
sleep_day$day_of_week <- format(as.Date(sleep_day$sleepdate), "%A")

#take a new view of sleep_day
head(sleep_day)

Now I will attempt to isolate date component from the daily_activity dataset and identify the day of the week for each date. First, I will identify the datatype for ActivityDate in daily_activity dataset. 

In [ ]:
#identify the datatype for ActivityDate in daily_activity
class(daily_activity$ActivityDate)

The next step involves removing the time component from the ActivityDate data and introducing a new column called New_date.

In [ ]:
#first step of extracting the date alone from the ActivityDate column. Data cleaning.
daily_activity$New_date <- sub(" \\d{2}:\\d{2}:\\d{2} (AM|PM)", "", daily_activity$ActivityDate)

#take a look at daily_activity
head(daily_activity)

Next, I will format the New_date column by creating a new column, New_date1, that will be in date datatype. I'll also identify the day of the week for each date entry by adding a day_of_week column. 

In [ ]:
#convert each entry in New_date to date datatype, store in new column, New_date1
daily_activity$New_date1<-as.Date(daily_activity$New_date, format = "%m/%d/%Y")

#identify the day of the week for each date in daily activity
daily_activity$day_of_week <- format(as.Date(daily_activity$New_date1), "%A")

#take a new look at daily_activity
head(daily_activity)

The next step of the analysis aggregates the data metrics from each dataset on the basis of the day of the week. Considering the number of dates, using day of the week would be a simpler means to generate insights. 
First, I'll analyze the sleep_day dataset. 

In [ ]:
#draw a bar chart showing the minutes slept per day of the week
#the first step to drawing the bar chart is converting the day_of_week column into a factor and specifying the order. This will ensure the graph is also in this order when it is drawn. 
sleep_day$day_of_week <- factor(sleep_day$day_of_week, 
                                levels = c("Monday", "Tuesday", "Wednesday", "Thursday", 
                                           "Friday", "Saturday", "Sunday"))
ggplot(sleep_day, aes(x = day_of_week, y = TotalMinutesAsleep)) + 
  geom_col()

From the graph above which plots based on the sum total of minutes for each day of the week, Bellabeat users get most minutes of sleep on Wednesday and least minutes of sleep on Monday. 

I will also calculate the median total minutes slept per day of week. 

In [ ]:
#Calculate the median of total minutes slept per day of the week
aggregate(sleep_day$TotalMinutesAsleep~sleep_day$day_of_week, FUN = median)

According to the table above, Sunday has the highest value for median of TotalMinutesAsleep and Friday has the lowest value. This table is more applicable in analyzing the sleep_day data than the column graph earlier created, and this is because while the column graph summed all the minutes per day, this table uses a more applicable aggregation tool i.e. median. 

I'll create a new table sleeptime_per_day to store the analysis from the table above. 

In [ ]:
#record median distribution of minutes slept by day of week in table 
sleeptime_per_day <- aggregate(sleep_day$TotalMinutesAsleep~sleep_day$day_of_week, FUN = median)

#view sleeptime_per_day
View(sleeptime_per_day)

For better comprehension, I'm going to rename the columns on sleeptime_per_day to weekday and MinAsleep.

In [ ]:
#rename the columns on sleeptime_per_day
sleeptime_per_day <- sleeptime_per_day %>% rename(weekday = `sleep_day$day_of_week`, MinAsleep = `sleep_day$TotalMinutesAsleep`)
#View sleeptime_per_day
View(sleeptime_per_day)

Next, I'll be introducing a new column called hours_asleep, which contains the hour conversion of MinAsleep. 

In [ ]:
#show sleeptime as hours on sleeptime_per_day
sleeptime_per_day$hours_asleep <- sleeptime_per_day$MinAsleep/60

#View sleeptime_per_day
View(sleeptime_per_day)

According to the National Sleep Foundation (NSF) and research from the Centers for Disease Control and Prevention (CDC), United States, the average person is supposed to sleep a recommended 8 hours every day. Taking that as a benchmark, we'll be deriving how many recommended hours/minutes of sleep are left per day. This will be stored in a new column on sleeptime_per_day called sleeptime_remain_in_mins. 

In [ ]:
#calculate hours of sleep remaining in minutes
sleeptime_per_day$sleeptime_remain_in_mins <- (8-sleeptime_per_day$hours_asleep)* 60 

#View sleeptime_per_day
View(sleeptime_per_day)

Next, I'm working on the daily_activity dataset to repeat the steps I just completed on sleep_day dataset to obtain the sleeptime_per_day table. 

First, I'll draw a graph of TotalSteps against the day_of_week. 

In [ ]:
#draw a bar chart showing the total steps walked per day of week 
#the first step to drawing the bar chart is converting the day_of_week column into a factor and specifying the order. This will ensure the graph is also in this order when it is drawn. 
daily_activity$day_of_week <- factor(daily_activity$day_of_week, 
                                levels = c("Monday", "Tuesday", "Wednesday", "Thursday", 
                                           "Friday", "Saturday", "Sunday"))
ggplot(daily_activity, aes(x = day_of_week, y = TotalSteps)) + geom_col()

I'm going to make a table that aggregates the median Total steps walked by day of the week. This table is more applicable to my analysis than the chart I just drew that sums all the steps walked per each day.

In [ ]:
#calculate the median of total steps walked per day of week
aggregate(daily_activity$TotalSteps~daily_activity$day_of_week, FUN = median)

I'm going to record the data in this table in a new dataset table and name it distancerec_per_day

In [ ]:
#record median distribution of total steps walked per by day of week in table 
distancerec_per_day <- aggregate(daily_activity$TotalSteps~daily_activity$day_of_week, FUN = median)

#View distancerec_per_day
View(distancerec_per_day)

For better comprehension, I'll rename the columns in distancerec_per_day to Weekday and TotalSteps.

In [ ]:
#rename the columns on distancerec_per_day
distancerec_per_day <- distancerec_per_day %>% rename(weekday = `daily_activity$day_of_week`, TotalSteps = `daily_activity$TotalSteps`)

#View distancerec_per_day
View(distancerec_per_day)

Given that the recommended number of steps to be taken per day is 10,000, I can derive how many steps are left per each day. This will be stored in a new column named Steps_left

In [ ]:
#calculate amount of steps remaining 
distancerec_per_day$Steps_left <- (10000-distancerec_per_day$TotalSteps)

#View distancerec_per_day
View(distancerec_per_day)

As a final step in my analysis, I will take important data columns from both sleeptime_per_day and distancerec_per_day to make a new table called combined.

In [ ]:
#join sleeptime_per_day and distancerec_per_day into one table 
combined <- data.frame(x = distancerec_per_day$weekday, y = distancerec_per_day$Steps_left, z = sleeptime_per_day$hours_asleep)

#View combined
View(combined)

Considering that all the data (i.e data from both sleep_day and daily_activity) were obtained from the same users, I postulate that y (Steps_left) should have a strong positive correlation with z (hours_asleep). This is because the app's tools measures activity by mobility. So if the users weren't moving, they were inactive. A lot of our inactive time is spent sleeping, so I believe that the more steps that are left to take per day, the more time people spent sleeping. 

Let's check to see if i'm correct. 

In [ ]:
#calculate the correlation between y(steps_left) and z(hours_asleep) on combined table 
cor(combined$y, combined$z)

The Pearson's Correlation coefficient is O.79 which is very strong. There is a strong likelihood that people spent inactive time sleeping. 

Let's go further to draw a graph showing this relationship. 

In [ ]:
#plot a graph showing how correlated steps_left and hours_asleep are. 
ggplot(combined, aes(x = y, y = z)) + geom_point() + geom_smooth(method = "lm", se = TRUE, color = "red") + labs(title = "Strong Positive Correlation (r = 0.79)", y = "X variable", z = "Y variable")

The following are my insights upon my analysis of data from Bellabeat users. 

i. People sleep the least on Fridays. Bellabeat needs to introduce new app features, programs or devices that remind people of the dangers of sleep deprivation and encourages them to sleep more on Fridays. If Bellabeat can provide data on the age of the users, analysis can be done to show the distribution of sleep per age range. This will also help determine the steps to take to improve sleep in the worst hit demographics. 

ii. People sleep most on Sundays, with Sunday being the only day when the Median sleep time is more than the recommended 8 hours. Bellabeat could set each person's Sunday sleep as a benchmark per person. If a user can match their Sunday sleep time for a stretch of days, Bellabeat provides rewards like gifts and discounts. Of course, this won't work for everyone because we're basing our analysis on the median value, but it would be beneficial to most people. 

iii. Bellabeat apps might also need to install reminders that make people walk more on the weekends, especially on Saturdays and Sundays which both have over 3000 on the Steps_left column. Walking is beneficial to health at all ages, and is one of the more easily completed fitness exercises. Getting people to take longer walks or move around more is a win for all sides. 
 

Similar analysis can also be made per person, rather than per day of the week. This would really help individual users because seeing their performance on simple health metrics like this can inform a user on the habits to inculcate and those to cut out. It would also help Bellabeat provide a more customisable service to each user. 

By every indication, Bellabeats health tracking tools are useful in monitoring lifestyle quirks. But without providing insight after analysis, they can't be utilized to full capacity. Bellabeat needs to provide these analysis to each customer, maybe even comparing individual performance to mean or median metrics. 

Lastly, I'm aware my analysis is far from robust. I ignored metrics like Calories, VeryActiveDistance, VeryActiveMinutes and SedentaryMinutes on the daily_activity dataset. And I could have also aggregated by ID or by date, rather than by day of the week. But I've only got so much time and I need to move on to other projects. I would do it on request though. I don't mind that. 

If you're still reading at this point, you're a champ who deserves the world. Thank you for reading through this borefest. I'll be in the comments for any feedback. 